# NLP HSE School

In [1]:
from pymystem3 import Mystem
import pandas as pd
import re
m=Mystem()
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [2]:
sentilex = pd.read_csv('RuSentiLex2017_revised_2utf.txt',
                       names=['word','part','initial','sentiment','source','amb1','amb2'],
                       skiprows=20,skipinitialspace=True,index_col=2)
sentilex['sentiment'] = sentilex['sentiment'].map({'negative':-1,'positive':1,'neutral':0,'positive/negative':0})

In [3]:
vocab = sentilex.to_dict(orient='index')

In [4]:

def read_file(n):
    file = pd.read_csv('Texts/art{}.txt'.format(n),sep='\n',names=['lines'])
    file.lines = file.lines.str.replace("{Author, Unknown} ","")
    
    ent_df = pd.read_csv('Texts/art{}.ann'.format(n),sep='\t',names=['ind','descr','ne'],header=None)
    ent_df['initial'] = ent_df['ne'].apply(lambda x: ("".join(m.lemmatize(x.lower()))).replace('\n',""))
    ent_df = ent_df.set_index(ent_df['ne'])
    syn_df = pd.read_json('Texts/art{}syn.json'.format(n))

    syn_df = syn_df.reset_index()
    def get_struct_data(x):
        sentl =[]
        pars=[]
        for text in x:
            for sent in text:
                sentl = sent.split(' ')
                to_app = [sentl[i] for i in range(len(sentl)) if i in([0,1,4,6,8,10,12])]
                to_app.append(m.lemmatize(to_app[2])[0])
                pars.append(to_app)
        return(pars)

    syn_df['syntax'] = syn_df['syntax'].apply(get_struct_data)
    
    syn_df['index']=syn_df['index'] +1
    file = file.reset_index().merge(syn_df)
    return(file.drop('text',1), ent_df.to_dict(orient='index'))


In [5]:
import pandas as pd
from sentiframes_proc import find_polar
table = pd.read_csv('sentiframes_df.csv')

lst = [['31', '42', 'бомбит', 'VERB', 'Aspect=Imperf|VerbForm=Inf|fPOS=VERB++', '2', 'nsubj', 'бомбить'],       
       ['78', '83', 'Сирии', 'NOUN', 'Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing|fPOS=NOUN++', '10', 'dobj', 'сирия'],
       ['107', '115', 'Румийлан', 'NOUN', 'Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing|fPOS=NOUN++', '15', 'appos', 'румийлан']]


find_polar(lst[0], lst[1], lst[2])


[('румийлан', 'сирия', -1), ('сирия', 'румийлан', -1)]

In [6]:
text_lines, ent_dict = read_file(1)

In [46]:
#получение данных из синтаксиса
def get_mutual_node(syntax,ent1,ent1pos,ent2,ent2pos):
    
    def get_node(e,p):

        e = e.split(" ")[0].split("-")[0].split(".")[0].split("»")[0]
        ent_dict = dict()
        for i in range(len(syntax)):
            if(e in syntax[i][2]):
                
                ent_dict[abs(int(syntax[i][1])-p)]=i
        
        return ent_dict[sorted(ent_dict)[0]]
    def get_route(i):
        route = list()
        for _ in range(len(syntax)):
            h=int(syntax[i][5])
            if(h ==-1): return route
            route.append(syntax[h])
            i=h
    ent_node1 = get_node(ent1,ent1pos)
    ent_node2 = get_node(ent2,ent2pos)
    
    route1= get_route(ent_node1)[::-1]
    route2= get_route(ent_node2)[::-1]
    if(len(route1)==0): return syntax[ent_node1],syntax[ent_node1],syntax[ent_node2]
    if(len(route2)==0): return syntax[ent_node2],syntax[ent_node1],syntax[ent_node2]
    
    obj=None
    for i in range(min(len(route1),len(route2))):
        
        if(route1[i][0]==route2[i][0]): obj=route1[i]
        else: break
     
    if(not obj):obj=route1[0]
    return [obj,syntax[ent_node1],syntax[ent_node2]]

In [8]:
import re
import nltk
from nltk import ngrams as ng
def get_sent_list(lemmas):
    sent_list = []
    string = "".join(lemmas)
    for i in reversed(range(1,min(11,len(lemmas)))):
        ngrams = ng(lemmas,i)
        for ngram in ngrams:
            subs = "".join(list(ngram))
            if(subs in vocab): 
                sent_list.append(vocab[subs]['sentiment'])
                lemmas = m.lemmatize(string.replace(subs,""))
    return sent_list

def sentiment_extraction(x,ent_dict):
    line = x['lines']
    syntax = x['syntax']
    ne_list = []
    for ne in ent_dict.keys():
        regex= re.compile(r"\b{}\b".format(re.escape(ne)),flags=re.UNICODE)
        for pos in ([m.start(0) for m in re.finditer(regex, line)]):
            ne_list.append((ne,pos))
    ne_pairs  = [(ne1,ne2) for ne1 in ne_list for ne2 in ne_list if ne1 != ne2]
    
    mutuals = list()
    polarities = list()
    mut_pol = list()
    for ((ne1, ne1_pos), (ne2,ne2_pos)) in ne_pairs:
        mut = get_mutual_node(syntax,ne1, ne1_pos, ne2, ne2_pos)
        if len(mut)>0:
            mutuals.append(mut)
        polar = find_polar(mut[0], mut[1], mut[2])
        if  polar!=None and len(polar)>0:
            polarities.append(polar)
        elif mut[0][len(mut[0])-1] in vocab:
            polarities.append([(ent_dict[ne1]['initial'],ent_dict[ne2]['initial'],vocab[mut[0][len(mut[0])-1]]['sentiment'])])
                   
    x['mutuals']=mutuals
    x['polarities']=polarities
    
    
    x['ne_pairs'] = ne_pairs
    
    x['ne_substrings'] = [(ne1,ne2,line[pos1+len(ne1):pos2]) for (ne1,pos1),(ne2,pos2) in ne_pairs if pos2>pos1+10]
    
    sent_list = list()
    for ne1,ne2,subs in x['ne_substrings']:
        sent_list.append((ne1,ne2,get_sent_list(m.lemmatize(subs))))
    x['sent_list'] = sent_list
    
    return(x)

In [48]:
#считывание и извлечение
text_lines, ent_dict = read_file(25)

text_lines = text_lines.apply(lambda x:sentiment_extraction(x,ent_dict),axis=1)
text_lines


,index,lines,syntax,mutuals,polarities,ne_pairs,ne_substrings,sent_list
0,1,Интервью с Каролиной Бороньской-Грыневецкой — ...,"[[0, 8, Интервью, NOUN, Animacy=Inan|Case=Nom|...","[[[0, 8, Интервью, NOUN, Animacy=Inan|Case=Nom...",[],"[((Каролиной Бороньской-Грыневецкой, 11), (Пол...","[(Каролиной Бороньской-Грыневецкой, Польского ...","[(Каролиной Бороньской-Грыневецкой, Польского ..."
1,2,Interia.pl: Осчастливит ли «Брексит» Владимира...,"[[0, 7, Interia, NOUN, Animacy=Inan|Case=Nom|G...","[[[8, 10, pl, NOUN, Degree=Pos|fPOS=ADV++, -1,...",[],"[((Владимира Путина, 37), (Путина, 47)), ((Пут...",[],[]
2,3,Каролина Бороньска-Грыневецка: Разумеется.,"[[0, 8, Каролина, NOUN, Animacy=Inan|Case=Nom|...",[],[],[],[],[]
3,4,"Есть такая закономерность, что любые раздоры, ...","[[0, 4, Есть, VERB, Aspect=Imp|Mood=Ind|Number...","[[[97, 103, радуют, VERB, Aspect=Imp|Mood=Ind|...","[[(европейский союз, кремль, 1)], [(кремль, ев...","[((Европейском Союзе, 73), (Кремль, 104)), ((К...","[(Европейском Союзе, Кремль, очень радуют )]","[(Европейском Союзе, Кремль, [1])]"
4,5,Он оказывает неформальную поддержку таким анти...,"[[0, 2, Он, PRON, fPOS=PRON++, 1, nsubj, он], ...","[[[58, 63, силам, NOUN, Animacy=Inan|Case=Dat|...",[],"[((Альтернатива для Германии, 70), (Национальн...","[(Альтернатива для Германии, Национальный фрон...","[(Альтернатива для Германии, Национальный фрон..."
5,6,"Путин заинтересован в ослаблении Европы, поско...","[[0, 5, Путин, NOUN, Animacy=Anim|Case=Nom|Gen...","[[[6, 19, заинтересован, ADJ, Degree=Pos|Gende...",[],"[((Путин, 0), (Европы, 33)), ((Путин, 0), (НАТ...","[(Путин, Европы, заинтересован в ослаблении )...","[(Путин, Европы, []), (Путин, НАТО, []), (Евро..."
6,7,Североатлантический альянс в последнее время у...,"[[0, 19, Североатлантический, ADJ, Case=Nom|De...","[[[45, 56, укрепляется, VERB, Aspect=Imp|Mood=...",[],"[((Североатлантический альянс, 0), (России, 84...","[(Североатлантический альянс, России, в после...","[(Североатлантический альянс, России, [1])]"
7,8,Между тем Евросоюз пребывает в глубоком кризис...,"[[0, 5, Между, ADP, fPOS=ADP++, 1, case, между...","[[[19, 28, пребывает, VERB, Aspect=Imp|Mood=In...",[],"[((Евросоюз, 10), (Кремле, 111)), ((Кремле, 11...","[(Евросоюз, Кремле, пребывает в глубоком криз...","[(Евросоюз, Кремле, [1, 0])]"
8,9,— Москва сможет перевести дух?,"[[0, 1, —, PUNCT, fPOS=PUNCT++,, 1, punct, —\n...",[],[],[],[],[]
9,10,— Ослабление ЕС может повлиять на перспективу ...,"[[0, 1, —, PUNCT, fPOS=PUNCT++,, 3, punct, —\n...","[[[16, 21, может, VERB, Aspect=Imp|Mood=Ind|Nu...",[],"[((ЕС, 13), (России, 86)), ((России, 86), (ЕС,...","[(ЕС, России, может повлиять на перспективу с...","[(ЕС, России, [0])]"


In [10]:
def get_summed_over_sent(sent_list):
    sdict = dict()
    for ne1,ne2,slist in sent_list:
        if (ne1,ne2) in sdict:sdict[(ent_dict[ne1]['initial'],ent_dict[ne2]['initial'])]+=sum(slist)
        else: sdict[(ent_dict[ne1]['initial'],ent_dict[ne2]['initial'])]=sum(slist) 
    return sdict
text_lines['summed'] =  text_lines['sent_list'].apply(get_summed_over_sent)
text_lines['summed']

0     {('сми', 'финляндия'): 0, ('сми', 'нато'): -1,...
1                                                    {}
2                                                    {}
3     {('сирия', 'румийлан'): 0, ('сирия', 'khorasan...
4                                                    {}
5     {('пентагон', 'сша'): 0, ('пентагон', 'игила')...
6     {('дамаск', 'сирия'): -2, ('дамаск', 'вашингто...
7     {('москва', 'дамаск'): 0, ('москва', 'белый до...
8     {('сирия', 'сша'): 0, ('сирия', 'вашингтон'): ...
9     {('сирия', 'иго'): -6, ('сирия', 'обама'): -1,...
10                     {('обама', 'ближний восток'): 2}
11    {('ракка', 'иго'): 0, ('вкс', 'ракка'): 0, ('в...
12                                                   {}
13                                                   {}
14                            {('дамаск', 'резах'): -1}
15    {('белый дом', 'москва'): -2, ('белый дом', 'д...
16    {('сми', 'сирия'): -1, ('сми', 'москва'): -1, ...
17          {('сша', 'дамаск'): 0, ('рф', 'дамас

In [11]:
def get_summed_over_polar(polar):
    sdict = dict()

    for p in polar:
        for sent_list in p:
            ne1,ne2,slist = sent_list
            if (ne1,ne2) in sdict:sdict[(ne1,ne2)]+=slist
            else: sdict[ne1,ne2]=slist 
    return sdict
text_lines['sum_por'] =  text_lines['polarities'].apply(get_summed_over_polar)
text_lines['sum_por']

0                                                    {}
1                                                    {}
2                                                    {}
3                                                    {}
4                                                    {}
5                                                    {}
6     {('дамаск', 'дамаск'): -2, ('сирия', 'вашингто...
7                                                    {}
8                                                    {}
9                                                    {}
10    {('обама', 'ближний восток'): 0, ('ближний вос...
11                                                   {}
12                                                   {}
13                                                   {}
14                                                   {}
15    {('белый дом', 'москва'): -1, ('белый дом', 'д...
16                                                   {}
17                                              

In [12]:
opinions = dict()
def get_summed(sent_list):
        for k in sent_list.keys():
            if k in opinions : opinions[k]+=sent_list[k]
            else: opinions[k]=sent_list[k]
            
_ = text_lines['summed'].apply(get_summed)
_ = text_lines['sum_por'].apply(get_summed)

df = pd.DataFrame([(ne1,ne2,opinions[(ne1,ne2)]) for (ne1,ne2) in opinions.keys()],columns=['ne1','ne2','sent'])
df

,ne1,ne2,sent
0,сми,финляндия,0
1,сми,нато,-1
2,сми,россия,-2
3,иран,нато,-1
4,иран,россия,-2
5,финляндия,нато,0
6,финляндия,россия,-2
7,нато,россия,-2
8,сирия,румийлан,0
9,сирия,khorasan,0


In [13]:
df['sent'] = df['sent'].apply(lambda x: 'negative' if x<0 else ('positive' if x>0 else None))


In [14]:
df[~df['sent'].isnull()].to_csv('Texts/1.csv',index=False,header=False)

In [50]:
#считывание и извлечение
from tqdm import tqdm
for i in tqdm(range(46,75)):
    if(i in [70]): continue
    print (i)
    text_lines, ent_dict = read_file(i)

    text_lines = text_lines.apply(lambda x:sentiment_extraction(x,ent_dict),axis=1)
    opinions = dict()
    text_lines['summed'] =  text_lines['sent_list'].apply(get_summed_over_sent)
    text_lines['sum_por'] =  text_lines['polarities'].apply(get_summed_over_polar)
    _ = text_lines['summed'].apply(get_summed)
    _ = text_lines['sum_por'].apply(get_summed)
    df = pd.DataFrame([(ne1,ne2,opinions[(ne1,ne2)]) for (ne1,ne2) in opinions.keys()],columns=['ne1','ne2','sent'])
    df['sent'] = df['sent'].apply(lambda x: 'negative' if x<0 else ('positive' if x>0 else None))
    df = df[df['ne1']!=df['ne2']]
    df[~df['sent'].isnull()].to_csv('Texts/{}.csv'.format(i),index=False,header=False)

  0%|          | 0/29 [00:00<?, ?it/s]

46


  3%|▎         | 1/29 [00:01<00:35,  1.26s/it]

47


  7%|▋         | 2/29 [00:02<00:31,  1.18s/it]

48


 10%|█         | 3/29 [00:02<00:25,  1.01it/s]

49


IndexError: ('list index out of range', 'occurred at index 61')

In [ ]:
text_lines

In [16]:
df

,ne1,ne2,sent
0,франк-вальтер штайнмайер (frank-walter steinmeier,нато,None
1,франк-вальтер штайнмайер (frank-walter steinmeier,штайнмайер,None
2,штайнмайер,нато,None
3,франк-вальтер штайнмайер,нато,None
4,франк-вальтер штайнмайер,штайнмайер,positive
5,нато,альянс,None
6,нато,польша,None
7,нато,статья 5 североатлантический договор,None
8,польша,альянс,None
9,польша,статья 5 североатлантический договор,None
